In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3' #CHANGE VERSION NUMBER HERE FROM LIST FROM WEBSITE!
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [2]:
 !wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-12 05:02:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.23MB/s    in 0.2s    

2023-01-12 05:02:03 (6.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
 from pyspark.sql import SparkSession
 spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [8]:
from pyspark.sql.functions import to_date, col

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes","total_votes","vine","verified_purchase"])

vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [11]:
#total votes equal or greater to 20 to avoid null and fine better reviews
cleaned_df = vine_df.filter("total_votes >= 20")

cleaned_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [38]:
#make new dataframe where helpful votes / total votes = 50% or more
#converting to Double Type
cleaned_df = cleaned_df.withColumn("total_votes",cleaned_df.total_votes.cast("double"))
cleaned_df = cleaned_df.withColumn("helpful_votes", cleaned_df.helpful_votes.cast("double"))

fifty_plus_df = cleaned_df.filter(cleaned_df.helpful_votes/ cleaned_df.total_votes >= 0.5 )


fifty_plus_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|         21.0|       34.0|   N|                N|
|R2CI0Y288CC7E2|          1|         21.0|       35.0|   N|                Y|
|R127WEQY2FM1T3|          1|        147.0|      175.0|   N|                Y|
|R2FJ94555FZH32|          2|         55.0|       60.0|   N|                N|
|R1U3AR67RE273L|          1|         51.0|       65.0|   N|                Y|
|R3PZOXA5X1U8KW|          4|         31.0|       36.0|   N|                N|
| R6KTC1OPIOIIG|          2|         19.0|       34.0|   N|                Y|
|R36O341WWXXKNP|          5|         28.0|       31.0|   N|                N|
|R10LZVBLQHBVJ0|          2|        151.0|      198.0|   N|                N|
|R1VR5GLGY1GE7N|          1|         49.0|       51.0|   N|     

In [22]:
#new DF where reviews are part of vine program
vine_dfYES = fifty_plus_df.filter(fifty_plus_df.vine == "Y")

vine_dfYES.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|         56.0|       63.0|   Y|                N|
|R10FO5UKKVZBK2|          3|         23.0|       23.0|   Y|                N|
| RM4KSGEOR7MU1|          5|         19.0|       24.0|   Y|                N|
| RG7VRMYLEXD23|          4|         22.0|       26.0|   Y|                N|
|R11O4YSCPSNL6L|          3|         20.0|       26.0|   Y|                N|
|R286MFBAJ8NPD6|          5|         46.0|       51.0|   Y|                N|
|R1JRR530H4COA2|          5|         22.0|       28.0|   Y|                N|
| RQ5WD90PUNBU9|          5|         21.0|       24.0|   Y|                N|
|R12648VHCQWUV9|          4|         21.0|       28.0|   Y|                N|
|R3KAW29CJ8L6DQ|          5|         17.0|       20.0|   Y|     

In [23]:
#new DF where reviews not part of the Vine program
vine_dfNO = fifty_plus_df.filter(fifty_plus_df.vine == "N")

vine_dfNO.show(10)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|         21.0|       34.0|   N|                N|
|R2CI0Y288CC7E2|          1|         21.0|       35.0|   N|                Y|
|R127WEQY2FM1T3|          1|        147.0|      175.0|   N|                Y|
|R2FJ94555FZH32|          2|         55.0|       60.0|   N|                N|
|R1U3AR67RE273L|          1|         51.0|       65.0|   N|                Y|
|R3PZOXA5X1U8KW|          4|         31.0|       36.0|   N|                N|
| R6KTC1OPIOIIG|          2|         19.0|       34.0|   N|                Y|
|R36O341WWXXKNP|          5|         28.0|       31.0|   N|                N|
|R10LZVBLQHBVJ0|          2|        151.0|      198.0|   N|                N|
|R1VR5GLGY1GE7N|          1|         49.0|       51.0|   N|     

In [31]:
#Determine the total number of reviews, the number of 5-star reviews,
 #and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

#Vine Program

vineTotal = vine_dfYES.select(vine_dfYES.review_id).count()
print("Total number of Vine reviews")
print(vineTotal)

#number of 5-star review
vine5 = vine_dfYES.filter(vine_dfYES.star_rating == "5").count()
print("total number of Vine 5-star reviews")
print(vine5)

#percentage
vinePercent = vine5/vineTotal
print("percentage of Vine five star reviews")
print(vinePercent)

Total number of Vine reviews
94
total number of Vine 5-star reviews
48
percentage of Vine five star reviews
0.5106382978723404


In [33]:
#Determine the total number of reviews, the number of 5-star reviews,
 #and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

#Non-Vine Program

noTotal = vine_dfNO.select(vine_dfNO.review_id).count()
print("Total number of non-Vine reviews")
print(noTotal)

#number of 5-star review
no5 = vine_dfNO.filter(vine_dfNO.star_rating == "5").count()
print("total number of non-Vine 5-star reviews")
print(no5)

#percentage
noPercent = no5/noTotal
print("percentage of non-Vine five star reviews")
print(noPercent)

Total number of non-Vine reviews
40471
total number of non-Vine 5-star reviews
15663
percentage of non-Vine five star reviews
0.38701786464381904
